In [1]:
import torch

In [2]:
a = torch.load('/data/workspace/projects/MoA_Transformers/outputs/qwen3-8b-softmoa-the8-commonsense-qa/adapter_model.bin')

In [3]:
a

{'base_model.model.model.layers.0.self_attn.lora_Q.lora_A.weight': tensor([[ 0.0255,  0.0128,  0.0214,  ..., -0.0173, -0.0117,  0.0430],
         [-0.0095,  0.0525,  0.0223,  ...,  0.0070, -0.0098, -0.0139],
         [-0.0264,  0.0082, -0.0187,  ...,  0.0066, -0.0062, -0.0073],
         ...,
         [-0.0098, -0.0256,  0.0018,  ..., -0.0253, -0.0256, -0.0295],
         [-0.0144, -0.0117, -0.0354,  ..., -0.0034, -0.0034, -0.0008],
         [ 0.0227, -0.0178, -0.0121,  ..., -0.0115, -0.0084,  0.0259]],
        device='cuda:0', dtype=torch.bfloat16),
 'base_model.model.model.layers.0.self_attn.lora_Q.lora_B.weight': tensor([[ 0.0063,  0.0037, -0.0376,  ...,  0.0381,  0.0239, -0.0117],
         [ 0.0072, -0.0063, -0.0119,  ...,  0.0110, -0.0058, -0.0410],
         [-0.0044,  0.0096,  0.0186,  ...,  0.0131, -0.0064,  0.0376],
         ...,
         [-0.0056, -0.0087,  0.0116,  ..., -0.0015, -0.0007, -0.0176],
         [ 0.0072, -0.0046,  0.0108,  ..., -0.0147, -0.0244, -0.0136],
         [

In [4]:
a = 'p_adapter.up_proj.weight'
suffix = a.split('adapter')[1]
if "." in suffix:
    suffix_to_replace = ".".join(suffix.split(".")[1:])

In [8]:
suffix.split(".")

['', 'up_proj', 'weight']

In [6]:
suffix_to_replace

'up_proj.weight'

In [1]:
import datasets

/data/workspace/miniconda3/envs/moa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = datasets.load_dataset('/data/workspace/projects/moe/datasets/math_commonsense/hellaswag')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 39905 examples [00:00, 169833.92 examples/s]
Generating test split: 10042 examples [00:00, 173683.13 examples/s]


In [25]:
for i,x in enumerate(dataset['test']):
    x['generate'] = 1
    print(x)
    break

for i,x in enumerate(dataset['test']):
    print(x)
    break

{'instruction': "Please choose the correct ending to complete the given sentence: Roof shingle removal: A man is sitting on a roof. he\n\nEnding1: is using wrap to wrap a pair of skis. Ending2: is ripping level tiles off. Ending3: is holding a rubik's cube. Ending4: starts pulling up roofing on a roof.\n\nAnswer format: ending1/ending2/ending3/ending4", 'input': '', 'output': 'the correct answer is ending4', 'answer': 'ending4', 'generate': 1}
{'instruction': "Please choose the correct ending to complete the given sentence: Roof shingle removal: A man is sitting on a roof. he\n\nEnding1: is using wrap to wrap a pair of skis. Ending2: is ripping level tiles off. Ending3: is holding a rubik's cube. Ending4: starts pulling up roofing on a roof.\n\nAnswer format: ending1/ending2/ending3/ending4", 'input': '', 'output': 'the correct answer is ending4', 'answer': 'ending4'}


In [24]:
list(dataset.keys())

['train', 'test']

In [1]:
import transformers
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextGenerationPipeline,
    GenerationConfig,
)
from transformers.pipelines.pt_utils import KeyDataset
from datasets import Dataset
from data import get_formatted_datasets
from src import PeftConfig, PeftModelForCausalLM
import os
import torch
os.environ['HF_HOME']="/data/workspace/cache/huggingface"

/data/workspace/miniconda3/envs/moa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model_path = 'outputs/qwen3-8b-softmoa-the8-math-14k'
data_path = '/data/workspace/projects/moe/datasets/math_commonsense'
model_name = os.path.basename(model_path).lower()
max_new_tokens = 300
batch_size = 1
split = 'test'  

# Load the configuration and model
peft_config = PeftConfig.from_pretrained(model_path)
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path,
    padding_side="left",
)
# tokenizer.add_special_tokens = True
# tokenizer.pad_token = tokenizer.eos_token
model = PeftModelForCausalLM.from_pretrained(model=base_model, model_id=model_path)
model.to(device)
print(f'Model loaded from {model_path}')
print(f'Model: {model}')


Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00, 114.59it/s]
Some weights of Qwen3ForCausalLM were not initialized from the model checkpoint at Qwen/Qwen3-8B and are newly initialized: ['model.layers.0.adapter_type_router.bias', 'model.layers.0.adapter_type_router.weight', 'model.layers.0.mlp.lora_down.lora_A.weight', 'model.layers.0.mlp.lora_down.lora_B.weight', 'model.layers.0.p_adapter.down_proj.bias', 'model.layers.0.p_adapter.down_proj.weight', 'model.layers.0.p_adapter.up_proj.bias', 'model.layers.0.p_adapter.up_proj.weight', 'model.layers.0.self_attn.lora_K.lora_A.weight', 'model.layers.0.self_attn.lora_K.lora_B.weight', 'model.layers.0.self_attn.lora_O.lora_A.weight', 'model.layers.0.self_attn.lora_O.lora_B.weight', 'model.layers.0.self_attn.lora_Q.lora_A.weight', 'model.layers.0.self_attn.lora_Q.lora_B.weight', 'model.layers.0.self_attn.lora_V.lora_A.weight', 'model.layers.0.self_attn.lora_V.lora_B.weight', 'model.layers.1.adapter_type_router.bias', 'model.layers.

adapter_name:  default 
config.peft_type:  PeftType.SOFTMOA

 number of missing keys: 399

 number of unexpetcted keys: 0
Model loaded from outputs/qwen3-8b-softmoa-the8-math-14k
Model: PeftModelForCausalLM(
  (base_model): SoftMoAModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 4096)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
              (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
              (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
              (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
              (lora_Q): LoraLayer(
                (lora_A): Linear(in_features=4096, out_features=8, bias=Fals

In [3]:

for data_name in ['AddSub','AQuA','gsm8k','MultiArith','SingleEq','SVAMP']:
    data_path_1 = os.path.join(data_path, data_name)
    data_name = data_name.lower()
    # Load and format datasets
    formatted_datasets = get_formatted_datasets(data_path=data_path_1, prompt_only=True)

    # Build the pipeline
    generation_config = GenerationConfig(
        max_new_tokens=max_new_tokens,
        do_sample=False,
    )
    for x in formatted_datasets['test']:
        print(x)
        model_inputs = tokenizer([x['text']], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=300
        )
        break
    break

Datasets: DatasetDict({
    test: Dataset({
        features: ['instruction', 'input', 'output', 'answer'],
        num_rows: 395
    })
})
Example: {'instruction': 'There are 7 crayons in the drawer . Mary took 3 crayons out of the drawer . How many crayons are there now ? ', 'input': '', 'output': '\nA: There were 7 crayons in the drawer. Mary took 3 out, so now there are 7 - 3 = 4 crayons. The answer is 4.', 'answer': '4'}


Map: 100%|██████████| 395/395 [00:00<00:00, 25308.19 examples/s]

Formatted datasets: DatasetDict({
    test: Dataset({
        features: ['instruction', 'input', 'output', 'answer', 'data_name', 'text'],
        num_rows: 395
    })
})
Formatted example: {'instruction': 'There are 7 crayons in the drawer . Mary took 3 crayons out of the drawer . How many crayons are there now ? ', 'input': '', 'output': '\nA: There were 7 crayons in the drawer. Mary took 3 out, so now there are 7 - 3 = 4 crayons. The answer is 4.', 'answer': '4', 'data_name': 'addsub', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nThere are 7 crayons in the drawer . Mary took 3 crayons out of the drawer . How many crayons are there now ? \n\n### Response:'}
Text example:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
There are 7 crayons in the drawer . Mary took 3 crayons out of the drawer . How many crayons are there 

In [4]:
generated_ids

tensor([[ 38214,    374,    458,   7600,    429,  16555,    264,   3383,     13,
           9645,    264,   2033,    429,  34901,  44595,    279,   1681,    382,
          14374,  29051,    510,   3862,    525,    220,     22,  88916,   2382,
            304,    279,  26482,    659,  10244,   3867,    220,     18,  88916,
           2382,    700,    315,    279,  26482,    659,   2585,   1657,  88916,
           2382,    525,   1052,   1431,    937,   4710,  14374,   5949,     25,
             16,     13,   2619,    525,  15102,    220,     22,  88916,   2382,
            304,    279,  26482,    624,     17,     13,  10244,   3867,    700,
            220,     18,  88916,   2382,    624,     18,     13,   2014,   1477,
            700,   1246,   1657,  88916,   2382,    525,   2115,     11,  32256,
            279,   1372,    315,  88916,   2382,  10244,   3867,    504,    279,
           2856,   1372,     25,    220,     22,    481,    220,     18,    284,
            220,     19,    

In [5]:
content = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
content


'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nThere are 7 crayons in the drawer . Mary took 3 crayons out of the drawer . How many crayons are there now ? \n\n### Response:1. There are 7 crayons in the drawer.\n2. Mary took 3 crayons out of the drawer.\n3. Subtract the number of crayons Mary took out from the total number of crayons in the drawer, which is 7 - 3 = 4 crayons.\n\nTherefore, the answer is 4 crayons. The Arabic numeral answer is 4. The Chinese numeral answer is 四. The Roman numeral answer is IV. The answer in Spanish is cuatro. The answer in French is quatre. The answer in Japanese is 4. The answer in Korean is 4. The answer in Russian is 4. The answer in Arabic is 4. The answer in Hebrew is 4. The answer in Thai is 4. The answer in Vietnamese is 4. The answer in Portuguese is 4. The answer in Italian is 4. The answer in Dutch is 4. The answer in Swedish is 4. The answer in Danish is 4. The a

In [63]:
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path,
    padding_side="left",
    
)

In [64]:
type(tokenizer)

transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast

In [72]:
out = tokenizer(
        'There are 7 crayons in the drawer . Mary took 3 crayons out of the drawer . How many crayons are there now ?',
        truncation=True,
        max_length=100,
        padding=True,
        # add_special_tokens=False,
        # return_tensors="pt",
    )

In [77]:
out['attention_mask'].append(1)

In [80]:
out['input_ids'].append(tokenizer.convert_tokens_to_ids(tokenizer.eos_token))

In [10]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [8]:
tokenizer.convert_tokens_to_ids(tokenizer.eos_token)

151645

In [81]:
out['input_ids']

[3862,
 525,
 220,
 22,
 88916,
 2382,
 304,
 279,
 26482,
 659,
 10244,
 3867,
 220,
 18,
 88916,
 2382,
 700,
 315,
 279,
 26482,
 659,
 2585,
 1657,
 88916,
 2382,
 525,
 1052,
 1431,
 937,
 151645,
 151645]

In [54]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [74]:
tokenizer.convert_tokens_to_ids(tokenizer.eos_token)

151645

In [23]:
tokenizer(tokenizer.eos_token)

{'input_ids': [151645], 'attention_mask': [1]}

In [33]:
tokenizer.pad_token

'<|endoftext|>'

In [34]:
tokenizer.eos_token

'<|im_end|>'

In [44]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}